In [21]:
import sys
sys.path.append("..")

from libs.InteractivePitchUI import *
from libs.data_manipulation import *
from libs.dim_reduction import *
from libs.feature_generation import *
from libs.clustering import *
from libs.convex_hull import *
from libs.alpha_shape import *
from libs.weight_generator import *
from libs.similar_movement import *

In [22]:
matches = compile_team_tracking_data_with_labels("../data", "Denmark", "../data/Labelled_ground_truths.csv")
df = extract_one_match(matches, 4)


In [23]:
func_control = lambda x:1
func_inverse = lambda x: 20/x
func_linear = lambda x: 200-x
func_exp = lambda x: np.exp(-x/20)

In [24]:
relevant_indices = df[df["Label"] == "breakthrough-wb"].index.to_numpy()
relevant_index = 1




In [25]:
print(relevant_indices)
print(df.loc[[relevant_indices[1]]][["match_name","Time [s]"]])

[154347 310614]
            match_name  Time [s]
310614  Denmark_Serbia    1038.0


In [26]:
similar_movement = find_similar_movement(df,relevant_indices[relevant_index],df.iloc[::48].index)

In [27]:
similar_movement_ranked = sorted(similar_movement, key=lambda x:x[0])
new_indices = [index for _,index in similar_movement_ranked][:20]


In [28]:
new_indices.append(relevant_indices[relevant_index])
new_indices.append(relevant_indices[relevant_index]+1)
new_df=df.loc[new_indices]

In [29]:
finished_indices_control = most_similar_with_wasserstein(relevant_indices[relevant_index], new_df,func_control,steps=1)
finished_indices_linear =most_similar_with_wasserstein(relevant_indices[relevant_index], new_df,func_linear,steps=1) 
finished_indices_inverse = most_similar_with_wasserstein(relevant_indices[relevant_index], new_df,func_inverse,steps=1)
finished_indices_exp = most_similar_with_wasserstein(relevant_indices[relevant_index], new_df,func_exp,steps=1)

        match_id  Time [s]  Time [s]_team half_team  minute_team  Period_team  \
310614         3   1038.00        1038.00        1H           17            1   
310615         3   1038.04        1038.04        1H           17            1   

        home_6_x  home_6_y  home_18_x  home_18_y  ...  away_25_y  home_26_x  \
310614 -8.633185 -5.876219 -17.163774  30.823956  ...        NaN        NaN   
310615 -8.669666 -5.864330 -17.246704  30.762590  ...        NaN        NaN   

        home_26_y  away_3_x  away_3_y  away_15_x  away_15_y            Label  \
310614        NaN       NaN       NaN        NaN        NaN  breakthrough-wb   
310615        NaN       NaN       NaN        NaN        NaN          Missing   

        Time[s]  Competition  
310614   1038.0   "EURO2024"  
310615      NaN          NaN  

[2 rows x 101 columns]
22
22
[[(-8.633185386657715, -5.876218795776367), (-17.163774490356445, 30.823955535888672), (28.132917404174805, 0.0011601612204685), (-25.134429931640625, 20.

In [30]:
finished_indices_control = np.array(finished_indices_control)
finished_indices_exp = np.array(finished_indices_exp)
finished_indices_linear = np.array(finished_indices_linear)
finished_indices_inverse = np.array(finished_indices_inverse)

# Perform the comparison
finished_indices_control = finished_indices_control[
    (finished_indices_control > relevant_indices[relevant_index] + 480) | 
    (finished_indices_control < relevant_indices[relevant_index] - 480)
]

# Perform the comparison
finished_indices_exp = finished_indices_exp[
    (finished_indices_exp > relevant_indices[relevant_index] + 480) | 
    (finished_indices_exp < relevant_indices[relevant_index] - 480)
]

# Perform the comparison
finished_indices_inverse = finished_indices_inverse[
    (finished_indices_inverse > relevant_indices[relevant_index] + 480) | 
    (finished_indices_inverse < relevant_indices[relevant_index] - 480)
]

# Perform the comparison
finished_indices_linear = finished_indices_linear[
    (finished_indices_linear > relevant_indices[relevant_index] + 480) | 
    (finished_indices_linear < relevant_indices[relevant_index] - 480)
]



In [31]:
PitchDisplay(df, [relevant_indices[relevant_index]])

In [32]:
PitchDisplay(df,finished_indices_control)



In [33]:
import datetime



result_df = df.loc[finished_indices_control][["match_name","Time [s]"]]
result_df["Time [s]"] = result_df["Time [s]"].apply(lambda x : str(datetime.timedelta(seconds=x)))

title_seperate = list(map(lambda x: str(x),df.loc[relevant_indices[relevant_index]][["match_name","Time [s]"]].to_numpy()))

folder_name = "Control_function" + title_seperate[0]+"_"+title_seperate[1]

generate_folder(folder_name)

saved_df = result_df.to_csv(folder_name + "/" +folder_name+"_ground_truth.csv")
result_df.sample(frac=1).to_csv(folder_name + "/" +folder_name+"_for_ordering_truth.csv")

Folder 'Control_functionDenmark_Serbia_1038.0' has been removed successfully.


In [34]:
result_df = df.loc[finished_indices_linear][["match_name","Time [s]"]]
result_df["Time [s]"] = result_df["Time [s]"].apply(lambda x : str(datetime.timedelta(seconds=x)))
title_seperate = list(map(lambda x: str(x),df.loc[relevant_indices[relevant_index]][["match_name","Time [s]"]].to_numpy()))

folder_name ="Linear_function" + title_seperate[0]+"_"+title_seperate[1]
generate_folder(folder_name)

result_df.to_csv(folder_name + "/" +folder_name+"_ground_truth.csv")
result_df.sample(frac=1).to_csv(folder_name + "/" +folder_name+"_for_ordering_truth.csv")

Folder 'Linear_functionDenmark_Serbia_1038.0' has been removed successfully.


In [35]:
result_df = df.loc[finished_indices_inverse][["match_name","Time [s]"]]
result_df["Time [s]"] = result_df["Time [s]"].apply(lambda x : str(datetime.timedelta(seconds=x)))
title_seperate = list(map(lambda x: str(x),df.loc[relevant_indices[relevant_index]][["match_name","Time [s]"]].to_numpy()))

folder_name ="Inverse_function" + title_seperate[0]+"_"+title_seperate[1]

generate_folder(folder_name)

saved_df = result_df.to_csv(folder_name + "/" +folder_name+"_ground_truth.csv")

result_df.sample(frac=1).to_csv(folder_name + "/" +folder_name+"_for_ordering_truth.csv")

Folder 'Inverse_functionDenmark_Serbia_1038.0' has been removed successfully.


In [36]:
result_df = df.loc[finished_indices_exp][["match_name","Time [s]"]]
result_df["Time [s]"] =result_df["Time [s]"].apply(lambda x : str(datetime.timedelta(seconds=x)))
title_seperate = list(map(lambda x: str(x),df.loc[relevant_indices[relevant_index]][["match_name","Time [s]"]].to_numpy()))

folder_name ="Exp_function" + title_seperate[0]+"_"+title_seperate[1]
generate_folder(folder_name)

saved_df = result_df.to_csv(folder_name + "/" +folder_name+"_ground_truth.csv")
result_df.sample(frac=1).to_csv(folder_name + "/" +folder_name+"_for_ordering_truth.csv")

Folder 'Exp_functionDenmark_Serbia_1038.0' has been removed successfully.
